In [6]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import random
import sklearn
import collections
from sklearn.model_selection import train_test_split
import json
import pylab
import tensorflow as tf
import tensorflow.math as tfmath
import tensorflow.keras as keras
from scipy.optimize import curve_fit
from tensorflow.keras import layers, Model
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import load_model
from sklearn.metrics import roc_curve, auc
import sklearn.metrics as sk

In [7]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [8]:
f=h5py.File('/eos/home-w/wsherman/AD_Work/n_tuples/40MHZ_data/background_for_training.h5','r')
Dataset=np.array(f["Particles"])

#for i, batch in enumerate(Dataset):
#  pt_sum=0
#  for j, particle in enumerate(Dataset[i,:,:]):
#    if particle[3]!=0:
#      pt_sum+=particle[0]
#  for j, particle in enumerate(Dataset[i,:,:]):
#    particle[0]=particle[0]/pt_sum
    
    
Data_Train=Dataset[0:2000000,:,0:3]
Data_Test=Dataset[2000001:3600000,:,0:3]
Data_Validate=Dataset[3600001:4000000,:,0:3]

Data_Train_Flat=np.reshape(Data_Train,(-1,57))
Data_Val_Flat=np.reshape(Data_Validate,(-1,57))

In [9]:
class Sampling(keras.layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


def make_encoder(input_dim,h_dim_1,h_dim_2,latent_dim):
    inputs=keras.Input(shape=(input_dim))
    #x=layers.BatchNormalization()(inputs)
    x=layers.Dense(h_dim_1, activation='relu')(inputs)
    x=layers.Dense(h_dim_2, activation='relu')(x)
    z_mean=layers.Dense(latent_dim, name='z_mean')(x)
    z_logvar=layers.Dense(latent_dim, name='z_log_var',kernel_initializer=keras.initializers.Zeros())(x)
    z=Sampling()([z_mean,z_logvar])
    encoder=keras.Model(inputs,[z_mean,z_logvar,z],name='encoder')
    return encoder




def make_decoder(input_dim,h_dim_1,h_dim_2,latent_dim):
    inputs=keras.Input(shape=(latent_dim))
    x=layers.Dense(h_dim_2, activation='relu')(inputs)
    x=layers.Dense(h_dim_1, activation='relu')(x)
    z=layers.Dense(input_dim)(x)
    decoder=keras.Model(inputs,z,name='decoder')
    return decoder

class VAE_Model(keras.Model):
    def __init__(self,encoder,decoder,**kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")
        self.beta=1

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]
    
    def set_beta(self,beta):
        self.beta=beta
    
    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            #making a masked loss function
            mask = K.cast(K.not_equal(data, 0), K.floatx())
            
            reconstruction_loss = tf.reduce_mean(tf.reduce_sum(keras.losses.mse(mask*data, mask*reconstruction)))
            reconstruction_loss *=(1-self.beta)

            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(kl_loss)
            kl_loss *=self.beta


            total_loss = reconstruction_loss + kl_loss
            
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reco_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

    def test_step(self, data):
        z_mean, z_log_var, z = self.encoder(data)
        
        reconstruction = self.decoder(z)
        mask = K.cast(K.not_equal(data, 0), K.floatx())
        reconstruction_loss = tf.reduce_mean(tf.reduce_sum(keras.losses.mse(mask*data, mask*reconstruction)))
        
        reconstruction_loss*=(1-self.beta)

        kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
        
        #KL loss changed abck to sum as in paper
        kl_loss = tf.reduce_sum(kl_loss)
        
        kl_loss *=self.beta
        
        total_loss = reconstruction_loss + kl_loss
        return {
            "loss": total_loss,
            "reco_loss": reconstruction_loss,
            "kl_loss": kl_loss,
        }

    def call(self, data):
        z_mean,z_log_var,x = self.encoder(data)
        reconstruction = self.decoder(x)
        return {
            "z_mean": z_mean,
            "z_log_var": z_log_var,
            "reconstruction": reconstruction
        }

        
    

In [11]:
#Here are the configurable parameters
STOP_PATIENCE = 8
LR_PATIENCE = 4

NUM_EPOCHS = 100
BATCH_SIZE = 1000

beta=0.83

#latent_space_sizes=[7,8,9,10,11,12]
#[0.001,0.1,0.2,0.3,0.4,0.5
beta_values=[0.5]
#[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
latent_sizes=[6,7,8]
#[1,2,3,4,5,6,7,8]
rep_num=10
#Here is the loop training several models with differently sized latent spaces
for b in beta_values:
    for s in latent_sizes:
            for n in range(0,rep_num):
                vae_enc=make_encoder(57,32,16,s)
                vae_dec=make_decoder(57,32,16,s)
                vae_40MHZ=VAE_Model(vae_enc,vae_dec)
                vae_40MHZ.set_beta(b)
                opt=keras.optimizers.Adam(learning_rate=0.001)
                vae_40MHZ.compile(optimizer=opt)



                early_stopping = EarlyStopping(patience=STOP_PATIENCE, restore_best_weights=True)
                reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=LR_PATIENCE, verbose=1)

                callbacks = [early_stopping, reduce_lr]



                history = vae_40MHZ.fit(x=Data_Train_Flat, validation_split=0.1,epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, callbacks=callbacks,shuffle=True)
                vae_40MHZ.save_weights(filepath='/eos/home-w/wsherman/AD_Work/ML_git_repo/AD_trigger_training/Miscellaneous Studies/Signal_Independant_Efficiency_Study/Models_for_Study/b_{}_s_{}_n{}/'.format(b,s,n),save_format='tf')

Epoch 1/100
1800/1800 [==============================] - 14s 5ms/step - loss: 548856.5820 - reco_loss: 543152.2282 - kl_loss: 1781.9402 - val_loss: 718408.8125 - val_reco_loss: 346.2984 - val_kl_loss: 718062.5000 - lr: 0.0010
Epoch 2/100
1800/1800 [==============================] - 9s 5ms/step - loss: 362.6258 - reco_loss: 267.8953 - kl_loss: 80.0966 - val_loss: 360631.7188 - val_reco_loss: 187.7482 - val_kl_loss: 360443.9688 - lr: 0.0010
Epoch 3/100
1800/1800 [==============================] - 9s 5ms/step - loss: 241.0576 - reco_loss: 191.9410 - kl_loss: 43.3885 - val_loss: 215295.0469 - val_reco_loss: 160.7582 - val_kl_loss: 215134.2812 - lr: 0.0010
Epoch 4/100
1800/1800 [==============================] - 9s 5ms/step - loss: 194.3176 - reco_loss: 164.0121 - kl_loss: 27.7391 - val_loss: 150949.7812 - val_reco_loss: 141.2512 - val_kl_loss: 150808.5312 - lr: 0.0010
Epoch 5/100
1800/1800 [==============================] - 9s 5ms/step - loss: 161.3955 - reco_loss: 139.7764 - kl_loss: 20.2

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



1800/1800 [==============================] - 9s 5ms/step - loss: 1975.4608 - reco_loss: 1864.3125 - kl_loss: 104.1353 - val_loss: 724596.5625 - val_reco_loss: 1004.7347 - val_kl_loss: 723591.8125 - lr: 0.0010
Epoch 14/100
1797/1800 [============================>.] - ETA: 0s - loss: 744.9342 - reco_loss: 656.3452 - kl_loss: 84.4427
Epoch 14: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
1800/1800 [==============================] - 9s 5ms/step - loss: 744.8184 - reco_loss: 656.2387 - kl_loss: 84.4287 - val_loss: 587708.3750 - val_reco_loss: 626.2408 - val_kl_loss: 587082.1250 - lr: 0.0010
Epoch 15/100
1800/1800 [==============================] - 9s 5ms/step - loss: 577.7378 - reco_loss: 502.4379 - kl_loss: 74.5190 - val_loss: 567136.7500 - val_reco_loss: 593.1060 - val_kl_loss: 566543.6250 - lr: 1.0000e-04
Epoch 16/100
1800/1800 [==============================] - 9s 5ms/step - loss: 115.8783 - reco_loss: 95.9252 - kl_loss: 18.2924 - val_loss: 114799.6484 - val_reco_

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



1800/1800 [==============================] - 9s 5ms/step - loss: 29.8246 - reco_loss: 26.6641 - kl_loss: 3.1532 - val_loss: 22430.2715 - val_reco_loss: 30.1933 - val_kl_loss: 22400.0781 - lr: 1.0000e-04
Epoch 55/100
1800/1800 [==============================] - 9s 5ms/step - loss: 29.5804 - reco_loss: 26.4455 - kl_loss: 3.1271 - val_loss: 22319.3574 - val_reco_loss: 29.9127 - val_kl_loss: 22289.4453 - lr: 1.0000e-04
Epoch 56/100
1800/1800 [==============================] - 9s 5ms/step - loss: 29.5394 - reco_loss: 26.4260 - kl_loss: 3.1108 - val_loss: 22223.0605 - val_reco_loss: 30.0840 - val_kl_loss: 22192.9766 - lr: 1.0000e-04
Epoch 57/100
1800/1800 [==============================] - 9s 5ms/step - loss: 29.3566 - reco_loss: 26.3084 - kl_loss: 3.0504 - val_loss: 21798.5703 - val_reco_loss: 29.7939 - val_kl_loss: 21768.7773 - lr: 1.0000e-04
Epoch 65/100
1800/1800 [==============================] - 9s 5ms/step - loss: 29.4153 - reco_loss: 26.3690 - kl_loss: 3.0444 - val_loss: 21728.4434 -

...

Epoch 1/100
1800/1800 [==============================] - 13s 5ms/step - loss: 6389.2530 - reco_loss: 6288.3963 - kl_loss: 92.4779 - val_loss: 467612.7812 - val_reco_loss: 278.5961 - val_kl_loss: 467334.1875 - lr: 0.0010
Epoch 2/100
1800/1800 [==============================] - 9s 5ms/step - loss: 227.9186 - reco_loss: 183.5358 - kl_loss: 36.9542 - val_loss: 222473.8750 - val_reco_loss: 187.6912 - val_kl_loss: 222286.1875 - lr: 0.0010
Epoch 3/100
1800/1800 [==============================] - 9s 5ms/step - loss: 31.7453 - reco_loss: 26.0617 - kl_loss: 5.5783 - val_loss: 48850.8438 - val_reco_loss: 72.7047 - val_kl_loss: 48778.1406 - lr: 0.0010
Epoch 11/100
1800/1800 [==============================] - 9s 5ms/step - loss: 32.3073 - reco_loss: 26.7509 - kl_loss: 5.3827 - val_loss: 43512.4414 - val_reco_loss: 32.4489 - val_kl_loss: 43479.9922 - lr: 0.0010
Epoch 12/100
1800/1800 [==============================] - 9s 5ms/step - loss: 33.3322 - reco_loss: 28.0780 - kl_loss: 5.1466 - val_loss: 413

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



1800/1800 [==============================] - 9s 5ms/step - loss: 31.6190 - reco_loss: 27.3184 - kl_loss: 4.2926 - val_loss: 34834.5820 - val_reco_loss: 34.9648 - val_kl_loss: 34799.6172 - lr: 0.0010
Epoch 24/100
1800/1800 [==============================] - 9s 5ms/step - loss: 31.4362 - reco_loss: 27.2319 - kl_loss: 4.1780 - val_loss: 34234.6719 - val_reco_loss: 26.9210 - val_kl_loss: 34207.7500 - lr: 0.0010
Epoch 25/100
1800/1800 [==============================] - 9s 5ms/step - loss: 29.9863 - reco_loss: 25.8868 - kl_loss: 4.0870 - val_loss: 34267.8633 - val_reco_loss: 30.1459 - val_kl_loss: 34237.7188 - lr: 0.0010
Epoch 26/100
1800/1800 [==============================] - 9s 5ms/step - loss: 28.7805 - reco_loss: 25.0497 - kl_loss: 3.7020 - val_loss: 29983.0273 - val_reco_loss: 35.4826 - val_kl_loss: 29947.5449 - lr: 0.0010
Epoch 34/100
1800/1800 [==============================] - 9s 5ms/step - loss: 29.6160 - reco_loss: 25.9596 - kl_loss: 3.6460 - val_loss: 29915.8672 - val_reco_loss: 

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



1800/1800 [==============================] - 9s 5ms/step - loss: 21.0961 - reco_loss: 18.1849 - kl_loss: 2.9069 - val_loss: 23617.7656 - val_reco_loss: 30.5810 - val_kl_loss: 23587.1855 - lr: 0.0010
Epoch 74/100
1800/1800 [==============================] - 9s 5ms/step - loss: 21.6745 - reco_loss: 18.7744 - kl_loss: 2.8948 - val_loss: 23437.2949 - val_reco_loss: 62.1779 - val_kl_loss: 23375.1172 - lr: 0.0010
Epoch 75/100
1800/1800 [==============================] - 9s 5ms/step - loss: 22.0540 - reco_loss: 19.1705 - kl_loss: 2.8860 - val_loss: 23476.0254 - val_reco_loss: 29.6624 - val_kl_loss: 23446.3633 - lr: 0.0010
Epoch 76/100
1800/1800 [==============================] - 9s 5ms/step - loss: 22.1759 - reco_loss: 19.2920 - kl_loss: 2.8769 - val_loss: 23317.6641 - val_reco_loss: 21.0043 - val_kl_loss: 23296.6602 - lr: 0.0010
Epoch 77/100
1800/1800 [==============================] - 9s 5ms/step - loss: 20.9942 - reco_loss: 18.1708 - kl_loss: 2.8197 - val_loss: 22877.4180 - val_reco_loss: 